<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/Web_scraping_levetech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4

In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from google.colab import files

# 親ページURL(指定：機械学習、データ分析、データマイニング)
url_pr = 'https://freelance.levtech.jp/project/search/?order=3&limit=3&sala=7&job%5B%5D=11&job%5B%5D=22&job%5B%5D=32&srchbtn=list_search&keyword=&tag_order='
# 子ページURL(ベース)
url_c_base = 'https://freelance.levtech.jp' 

html_doc = requests.get(url_pr).text
 
# データフレームを定義
columns = ["案件名","職務"]
df = pd.DataFrame(columns=columns)
 
# BeautifulSoupでスクレイピング
soup = BeautifulSoup(html_doc, 'html.parser')
# ラッピング単位のクラスを取得
contents  = soup.find_all("div", {"class": "prjHead"})
 
for content in contents:
    content_data = content.find("h3", {"class": "prjHead__ttl"})

    #新着(New) or 広告(CHECK)
    chk_ad = content_data.find("span")

    if(chk_ad is not None):
        if(chk_ad.text == "New"):
            flg = "OK"
        else:
            flg = "NG"
    else:
        flg = "OK"
        
    if(flg == "OK"):
        #案件名
        content_name = content_data.text
        content_name = content_name.replace("のエンジニア求人・案件", "")

        #子ページリンク先取得
        content_website = content_data.a.get("href")

        html_c_doc = requests.get(url_c_base + content_website).text
        soup_c = BeautifulSoup(html_c_doc, 'html.parser')
        # 子ページ内のクラスを取得
        contents_c  = soup_c.find("div", {"class": "pjtDetail__row"})

        #職務内容
        #(pタグ2個目が対象の要素)
        job_detail = contents_c.find_all("p")[1]
        job_detail = str(job_detail).replace("\r", "")
        job_detail = str(job_detail).replace("\n", "")
        job_detail = str(job_detail).replace("<p>", "")
        job_detail = str(job_detail).replace("</p>", "")
        job_detail = str(job_detail).replace("<br/>", "")
        job_detail = str(job_detail).strip()

        # 各データをデータフレームに格納
        se = pd.Series([content_name, job_detail], columns)
        df = df.append(se, columns)

# 収集したデータをCSV形式で保存
filename = "ai_list.csv"
df.to_csv(filename, encoding = 'utf-8')
files.download(filename)